In [38]:
import tensorflow as tf

def image_load(path):
    image = tf.io.read_file(path)
    image_tensor=tf.image.decode_jpeg(image,channels=3)
    image_tensor=tf.image.rgb_to_grayscale(image_tensor)
    image_tensor=tf.cast(image_tensor,tf.float32)
    image_tensor=tf.squeeze(image_tensor)
    image_tensor=image_tensor/255.
    return image_tensor

img = image_load('/tmp/capae/1-161149367113-0-808,jpg')
print(img.shape)
print(img)


(20, 15)
tf.Tensor(
[[1.         0.9098039  0.972549   0.9529412  0.9411765  0.88235295
  0.91764706 0.972549   0.70980394 0.6509804  1.         0.7490196
  0.7294118  0.99607843 0.7647059 ]
 [0.9411765  0.89411765 1.         0.87058824 0.9843137  0.9529412
  1.         0.31764707 0.42352942 0.93333334 0.67058825 0.972549
  0.8509804  0.78039217 0.9098039 ]
 [0.92156863 1.         0.8901961  0.90588236 0.3372549  0.30588236
  0.28627452 0.3529412  0.3647059  0.8901961  0.95686275 0.96862745
  1.         0.9411765  0.96862745]
 [0.9372549  0.8901961  0.95686275 0.37254903 0.2901961  0.35686275
  0.39607844 0.24313726 0.20392157 0.9490196  0.9647059  0.6901961
  0.99215686 0.90588236 0.8352941 ]
 [0.99215686 0.93333334 0.89411765 0.37254903 0.3019608  0.9411765
  0.9529412  0.27450982 0.3882353  0.93333334 0.92156863 1.
  0.69803923 1.         0.92156863]
 [0.8666667  0.92156863 0.9882353  0.9019608  0.85882354 0.8666667
  0.9411765  0.31764707 0.29803923 0.9254902  0.9411765  0.8666667


In [39]:
alllabels = "0123456789+x"
def label_to_index(label):
    for i in range(len(alllabels)):
        if alllabels[i] == label:
            return float(i)

print(label_to_index('3'))
print(label_to_index('x'))

def index_to_label(idx):
    return alllabels[idx]

print(index_to_label(0))
print(index_to_label(3))

3.0
11.0
0
3


In [47]:

import numpy as np

image_dataset_perelement_numpy=[]
image_dataset_labels=[]
for root, dirs, files in os.walk("/tmp/capae", topdown=False):
    image_count=len(files)
    for name in files:
        filepath = os.path.join(root, name)
        img = image_load(filepath)
        image_dataset_perelement_numpy.append(np.array(img))
        image_dataset_labels.append(label_to_index(name[0]))

image_dataset=np.array(image_dataset_perelement_numpy)
all_labels=np.array(image_dataset_labels)

print(image_count)
print(image_dataset.shape, all_labels.shape)

522
(522, 20, 15) (522,)


In [48]:

test_count=int(image_count*0.2)
train_count=image_count-test_count

train_image=image_dataset[test_count:train_count]
train_label=all_labels[test_count:train_count]
test_image=image_dataset[:test_count]
test_label=all_labels[:test_count]

print(train_image.shape, train_label.shape)
print(test_image.shape, test_label.shape)

(314, 20, 15) (314,)
(104, 20, 15) (104,)


In [49]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(20, 15)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(12, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
model.fit(train_image, train_label, epochs=15)

model.evaluate(test_image,  test_label, verbose=2)


Epoch 1/15
10/10 [==============================] - 0s 2ms/step - loss: 2.4244 - accuracy: 0.2580
Epoch 2/15
10/10 [==============================] - 0s 2ms/step - loss: 1.6439 - accuracy: 0.5382
Epoch 3/15
10/10 [==============================] - 0s 2ms/step - loss: 1.2358 - accuracy: 0.7261
Epoch 4/15
10/10 [==============================] - 0s 2ms/step - loss: 0.9612 - accuracy: 0.7930
Epoch 5/15
10/10 [==============================] - 0s 1ms/step - loss: 0.7758 - accuracy: 0.8503
Epoch 6/15
10/10 [==============================] - 0s 2ms/step - loss: 0.5660 - accuracy: 0.9204
Epoch 7/15
10/10 [==============================] - 0s 2ms/step - loss: 0.4497 - accuracy: 0.9650
Epoch 8/15
10/10 [==============================] - 0s 2ms/step - loss: 0.3512 - accuracy: 0.9809
Epoch 9/15
10/10 [==============================] - 0s 2ms/step - loss: 0.2945 - accuracy: 0.9904
Epoch 10/15
10/10 [==============================] - 0s 2ms/step - loss: 0.2244 - accuracy: 1.0000
Epoch 11/15
10/10 [

[0.09036552160978317, 1.0]

In [51]:
model.save('arithmetic-001.h5')


In [52]:

def topResult(result):
    topIdx = 0
    for i in range(len(result)):
        if result[i] > result[topIdx]:
            topIdx=i
    return index_to_label(topIdx)
    # print(topIdx, index_to_label(topIdx), result[topIdx])
 
procmodel = tf.keras.models.load_model("arithmetic-001.h5")

imgdata=image_load('/tmp/capae/3-120303536696-0-797,jpg')

result=procmodel.predict(np.array([imgdata,]))

# for v in np.nditer(np.sort(result), order='C'):
#     print(v)
# print(len(np.nditer(result, order='C')))
result = result[0].tolist()
topResult(result)
# for i in range(len(result)):
#     # print(i)
#     print(i, index_to_label(i), result[i])


'3'

In [53]:
def predict(imgs):
    imgdatas = []
    for img in imgs:
        imgdatas.append(image_load(img))
    result = procmodel.predict(np.array(imgdatas))
    resultStr = ''
    for i in range(len(imgs)):
        resultStr += topResult(result[i].tolist())
    return resultStr

res = predict([
    '/tmp/capae/5-129787508987-2-799,jpg',
    '/tmp/capae/4-120693185287-2-798,jpg',
    '/tmp/capae/6-154968900543-2-804,jpg',
    '/tmp/capae/7-085328446320-2-788,jpg',
    '/tmp/capae/9-056013241349-2-843,jpg',
    '/tmp/capae/x-195631212326-1-818.jpg',
    '/tmp/capae/x-082466455795-1-787,jpg',])
print(res)

# curl -F'count=1' -F 'file0=@"/tmp/capae/2-033622432474-0-778.jpg";filename="dqf_gng_"' http://localhost:5000/

546x
